<a href="https://colab.research.google.com/github/kzaho/DataScienceDiaries/blob/main/colab/question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q langchain==0.0.279
!pip install -q openai==0.28.0
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00

In [15]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# this won't work
# with open('/content/drive/MyDrive/LLM/token.gdoc') as f:

with open('/content/drive/MyDrive/cognitive_test_generator_with_KZ/LLM/token.txt') as f:
    api_key = f.readline()

In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(openai_api_key=api_key, model_name='gpt-4-0613')
llm = ChatOpenAI(openai_api_key=api_key, model_name='gpt-3.5-turbo')

In [18]:
def get_cat4_test_q_prompt(n_questions=5, user_instruction=""):

    ordered_instructions = [
        f"Generate JSON with list of {n_questions} questions.",
        """Each questions is a dict with such keys:
        question_id (string), question (string), options (list of strings), correct_answer (string), explanation (string).""",
        "Each question should only contain one correct answer. Incorrect answers should be plausible but incorrect.",
        "Provide explanation for the correct answer.",
        """Format the output as in the example:

        {{
            "questions": [
                {{
                    "question_id": 1,
                    "question": "Which one of the following is not a bird?",
                    "options": ["🦅", "🦉", "🦚", "🐱"],
                    "correct_answer": "🐱",
                    "explanation": "birds have wings, 🐱 does not"
                }},
                {{
                    "question_id": 2,
                    "question": "Which animal is known to love eating bananas?",
                    "options": ["🐶", "🐱", "🦓", "🐒"],
                    "correct_answer": "🐒",
                    "explanation": "it is hard for other animals to grasp a banana, so they usually eat other food"
                }}
            ]
        }}

        """,
        """Only use the following context (if exists) to come up with the questions:
        {text}""",
    ]

    prompt_template = "Abide by instructions.\n"
    instructions_counter = 1
    if len(user_instruction):
        prompt_template += f"Instruction {instructions_counter}. {user_instruction}\n"
        instructions_counter += 1

    for instruction in ordered_instructions:
        prompt_template += f"Instruction {instructions_counter}. {instruction}\n"
        instructions_counter += 1


    prompt_template += "\nJSON:"

    main_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

    refine_template = (
        "Your job is to refine a quiz with single-choice questions for children.\n"
        "We have provided an existing set of questions in JSON: {existing_answer}\n"
        "Here is some more context (if exists) that might help.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new context, refine the original set of questions.\n"
        "If the context isn't useful, return the original set of questions.\n"
        "Only respond with a single JSON.\n"
        "Here is the refined quiz in JSON format based on the provided context:"
    )

    refined_prompt = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    return main_prompt, refined_prompt

In [19]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Question(BaseModel):
    question_id: str = Field(..., description="ID of the question")
    question: str = Field(..., description="The question text")
    options: List[str] = Field(..., description="List of answer options")
    correct_answer: str = Field(..., description="The correct answer")
    explanation: str = Field(..., description="Explanation for the correct answer")

class Quiz(BaseModel):
    questions: List[Question] = Field(..., description="List of questions")

parser = PydanticOutputParser(pydantic_object=Quiz)

In [20]:
from langchain.output_parsers import OutputFixingParser
import json


def gen_cat4_questions(user_instruction,  n_questions=5, docs = [""]):

  main_prompt, refined_prompt = get_cat4_test_q_prompt(
      n_questions=n_questions,
      user_instruction=user_instruction
  )

  chain = load_summarize_chain(
      llm=llm,
      chain_type="refine",
      return_intermediate_steps=True,
      question_prompt=main_prompt,
      refine_prompt=refined_prompt,
  )

  docs = [Document(page_content=doc_text) for doc_text in docs]

  output = chain({"input_documents": docs}, return_only_outputs=True)

  try:
    parsed_output = json.loads(output["output_text"])
  except:
    misformatted = output["output_text"]
    new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
    quiz = new_parser.parse(misformatted)
    parsed_output = [{
      "question_id": j.question_id,
      "question": j.question,
      "options": j.options,
      "correct_answer": j.correct_answer,
      "explanation": j.explanation
    } for j in quiz.questions]

  return parsed_output

In [21]:
from bs4 import BeautifulSoup
import requests

# Fetch the HTML content from a website (replace the URL with the one you're interested in)
def get_content_pairs(url = "https://www.gutenberg.org/files/21/21-h/21-h.htm"):
  response = requests.get(url)

  # Parse the HTML content
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find all h2 tags
  h2_tags = soup.find_all('h2')

  # Initialize an empty list to store h2 and the content between them
  h2_content_pairs = []

  # Loop through each h2 tag to find the content between them
  for i in range(len(h2_tags) - 1):
      h2_start = h2_tags[i]
      h2_end = h2_tags[i + 1]

      content = []
      for elem in h2_start.find_next_siblings():
          if elem == h2_end:
              break
          if elem.name == 'p':
              content.append(elem.text.strip())

      h2_content_pairs.append({
          'h2': h2_start.text.strip(),
          'content': ' '.join(content)
      })

  # Print the pairs
  # for pair in h2_content_pairs[:3]:
  #     print(f"H2: {pair['h2']}")
  #     print(f"Content: {pair['content']}")
  #     print("---")

  return h2_content_pairs

In [22]:
content_pairs = get_content_pairs()[1:]
content_dict = {j['h2']:j['h2']+'\n'+j['content'] for j in content_pairs}
content_keys = [j['h2'] for j in content_pairs]

In [23]:
import time

# Record the start time
start_time = time.time()

# Your code here
parsed_output = gen_cat4_questions(
    user_instruction="Create a Verbal Reasoning question for 11+ United Kingdom exam.",
    n_questions=5,
    docs=[content_dict[content_keys[0]]]
)

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 48.95587348937988 seconds


suggestions for GRADIO




and workflow
1. reference this source file [/dir source]
2. ref /prompt_template  [template1.txt] [file open]

1a. load the prompt: Create a Verbal Reasoning {/source/Alpha1.txt.var.x}question {/source/Beta1.txt.var.x}}using Animal emojis for 11+ United Kingdom exam.


3. write file to /source [google sheet]



For user to be able to iterate over a google spreadsheet of data line by line and pass the results to a prompt that can handle variables and then LLM write the results to the same sheet. (simple) GRADIO idea: limit the number of lines being passed at anyone time.


Validate the JSON output with https://jsonlint.com/

In [27]:
import gradio as gr
import json

# Your original function
def my_function(input_text, slider_value, content_key, **kwargs):
    parsed_output = gen_cat4_questions(
        user_instruction=input_text,
        n_questions=slider_value,
        docs=[content_dict[content_key]]
    )

    return json.dumps(parsed_output)

iface = gr.Interface(
    fn=my_function,  # Function to be triggered
    inputs=[
        gr.Textbox(
            label="Promt",
            value="Create a Verbal Reasoning question for 11+ United Kingdom exam."
        ),
        gr.Slider(minimum=3, maximum=20, default=5, step=1),
        gr.Dropdown(
            content_keys, label="AESOP’S FABLES", info="By Aesop, Translated by George Fyler Townsend"
        )
    ],
    outputs="json",
    live=False
)

iface.launch(debug=True, share=True)

<ipython-input-27-65e51a555b8f>:21: GradioUnusedKwargWarning: You have unused kwarg parameters in Slider, please remove them: {'default': 5}
  gr.Slider(minimum=3, maximum=20, default=5, step=1),


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7a9b08cc1ca5b56822.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7a9b08cc1ca5b56822.gradio.live


In [29]:
with open("quiz_data.json", "w") as f:
    json.dump(parsed_output, f)

In [31]:
import gradio as gr
import json

def check_answers(*answers):
    with open("quiz_data.json", "r") as f:
        quiz_data = json.load(f)

    results = []
    for i, answer in enumerate(answers):
        question = quiz_data["questions"][i]
        if answer == question["correct_answer"]:
            results.append(f"Question {i+1}: Correct")
        else:
            results.append(f"Question {i+1}: Incorrect")
    return "\n".join(results)

# Load quiz_data.json to generate the quiz interface
with open("quiz_data.json", "r") as f:
    quiz_data = json.load(f)

quiz_inputs = [
    gr.Dropdown(
        choices=quiz_data["questions"][i]["options"],
        label=quiz_data["questions"][i]["question"]
    )
    for i in range(len(quiz_data["questions"]))
]

iface2 = gr.Interface(
    fn=check_answers,
    inputs=quiz_inputs,
    outputs="text",
    live=False
)

iface2.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/source/common phrasal verbs 1.

---

1. open the file common phrasal verbs
2. read the first line
3. open the prompt common phrasal verbs.txt
pass the line to the prompt
4